In [1]:
sc

<SparkContext master=spark://DESKTOP-K9GD4CQ.localdomain:7077 appName=PySparkShell>

In [2]:
spark

In [3]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, LongType

In [4]:
sqlContext = SQLContext(sc)

In [5]:
schema = StructType().add("action", StringType()) \
.add("count", LongType()) \
.add("placename", StringType()) \
.add("placetags", ArrayType(StringType())) \
.add("username", StringType())

In [6]:
df = sqlContext.read \
        .format("org.elasticsearch.spark.sql") \
        .schema(schema) \
        .option("es.nodes","my-es-server:9200") \
        .option("es.read.field.include", "action, count, placename, username, placetags") \
        .load("django_log_counts_tags_renamed")

In [7]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- count: long (nullable = true)
 |-- placename: string (nullable = true)
 |-- placetags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- username: string (nullable = true)



In [8]:
df.registerTempTable("logs")

In [17]:
output = sqlContext.sql("DESCRIBE logs").show()

+---------+-------------+-------+
| col_name|    data_type|comment|
+---------+-------------+-------+
|   action|       string|   null|
|    count|       bigint|   null|
|placename|       string|   null|
|placetags|array<string>|   null|
| username|       string|   null|
+---------+-------------+-------+



In [18]:
output = sqlContext.sql("SELECT * FROM logs WHERE username = 'testuser123'").show()

+------+-----+-------------------------+------------------------------+-----------+
|action|count|                placename|                     placetags|   username|
+------+-----+-------------------------+------------------------------+-----------+
|BROWSE|    1|               칠기마라탕|[, 먹어야산다, 중식, 마라탕...|testuser123|
|BROWSE|    1|                   자스민|[, 먹어야산다, 중식, 중화요...|testuser123|
|BROWSE|    1|            비야 고대본점|[, 먹어야산다, 한식, 부대찌...|testuser123|
|BROWSE|   23|                 고른햇살|[, 먹어야산다, 분식, 정대후...|testuser123|
|BROWSE|    5|                 특별식당|[, 먹어야산다, 일식, 돈부리...|testuser123|
|BROWSE|    3|               연어이야기|[, 먹어야산다, 제기동, 정문...|testuser123|
|BROWSE|    1|                 해곤선생|[, 먹어야산다, 중식, 중화요...|testuser123|
|BROWSE|    1|     시그니처 키친 (뚝닭)| [, 먹어야산다, 한식, 퓨전,...|testuser123|
|BROWSE|    1|달링스테이크 고려대가족점|[, 먹어야산다, 경양식, 스테...|testuser123|
|BROWSE|    5|                 어흥식당|[, 먹어야산다, 양식, 스테이...|testuser123|
|BROWSE|    6|                 어흥식당|[, 먹어야산다, 양식, 스테이...|testu